<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Implement-Naive-Bayes-From-Scratch" data-toc-modified-id="Implement-Naive-Bayes-From-Scratch-1">Implement Naive Bayes From Scratch</a></span></li><li><span><a href="#Bayes'-Theorem" data-toc-modified-id="Bayes'-Theorem-2">Bayes' Theorem</a></span></li><li><span><a href="#Learning-Outcomes" data-toc-modified-id="Learning-Outcomes-3">Learning Outcomes</a></span></li><li><span><a href="#A-Recipe-for-Naive-Bayes-Classification" data-toc-modified-id="A-Recipe-for-Naive-Bayes-Classification-4">A Recipe for Naive Bayes Classification</a></span></li><li><span><a href="#Acquire-data-&amp;-preprocess" data-toc-modified-id="Acquire-data-&amp;-preprocess-5">Acquire data &amp; preprocess</a></span></li><li><span><a href="#Calculate-document-class-priors" data-toc-modified-id="Calculate-document-class-priors-6">Calculate document class priors</a></span></li><li><span><a href="#Calculate-conditional-probabilities-of-each-word-for-each-class" data-toc-modified-id="Calculate-conditional-probabilities-of-each-word-for-each-class-7">Calculate conditional probabilities of each word for each class</a></span></li><li><span><a href="#Given-a-new-document-without-a-label,--calculate-the-proportional-probabilities-for-each-class" data-toc-modified-id="Given-a-new-document-without-a-label,--calculate-the-proportional-probabilities-for-each-class-8">Given a new document without a label,  calculate the proportional probabilities for each class</a></span></li><li><span><a href="#Pick-the-winning-class" data-toc-modified-id="Pick-the-winning-class-9">Pick the winning class</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-10">Summary</a></span></li><li><span><a href="#Bonus-Material" data-toc-modified-id="Bonus-Material-11">Bonus Material</a></span></li><li><span><a href="#Further-Study" data-toc-modified-id="Further-Study-12">Further Study</a></span></li></ul></div>

Implement Naive Bayes From Scratch
-----

Bayes' Theorem
------

$$ P(A | B) = \frac{P(B|A)P(A)}{P(B) $$

![](images/bayes_rule.png)

Learning Outcomes
------

__By The End Of This Notebook You Should Be Able To:__

1. Write idiomatic Python to model data. 
1. Implement Naive Bayes in pure Python.

------
A Recipe for Naive Bayes Classification
-------

1. Acquire labeled training data
1. With training data:
    1. Calculate document class priors
    1. Calculate conditional probabilities of each word for each class
1. With new data:
    1. Calculate the proportional probabilities for each class of new document
    1. Pick the winning class

Acquire data & preprocess
-----

In [74]:
reset -fs

```python
# Let's make a data class to hold our data
data = LabeledTextData(id_num=42, label='cat', words="🐱 🐱 🐈 🐶".split())
```

In [75]:
class LabeledTextData:
    def __init__(self, id_num, label, words):
        self.id_num = id_num
        self.label = label 
        self.words = words

In [76]:
data = LabeledTextData(id_num=42, label='cat', words="🐱 🐱 🐈 🐶".split())

Our `LabeledTextData` class is mostly data. We can use `dataclass` (new to Python 3.7) to simplify our code.

In [77]:
from dataclasses import dataclass

In [78]:
@dataclass
class LabeledTextData:
    id_num: int
    label: str
    words: list

In [79]:
train = [LabeledTextData(42, 'cat',  "🐈 🐯 🐱 🐩 🐱".split()),
         LabeledTextData(43, 'dog',  "🐶 🐶 🐈 🐶 🐩 🐈 🐶 🐶".split()),
         LabeledTextData(45, 'cat',  "🐈 🐈 🐯 🐶 🐈".split()),
         LabeledTextData(45, 'cat',  "🐈 🐈 🐈".split()),
         LabeledTextData(48, 'dog',  "🐶 🐶 🐯 🐈 🐩 🐱 🐩 🐶 🐩 🐶 ".split()),
        ]

Calculate document class priors
---- 

$$P(c) = \frac{N_c}{N}$$

In [80]:
# What labels are we dealing with?
labels = {doc.label for doc in train}
labels

{'cat', 'dog'}

In [81]:
# How many documents are dealing with?
n_docs = len(train)
n_docs

5

In [82]:
from collections import defaultdict

In [83]:
# For each label, find the probability of baseline occurance
class_counts = defaultdict(int)

# Find the count of each category
for doc in train:
    class_counts[doc.label] += 1

# Convert them to probabilities
doc_priors = {label: count/n_docs for label, count in class_counts.items()}
    
print(*doc_priors.items(), sep='\n')

('cat', 0.6)
('dog', 0.4)


Calculate conditional probabilities of each word for each class
-----

In [84]:
# Get all tokens, aka the vocabulary
vocab = []

for doc in train:
    vocab.extend(doc.words)
    
print("Vocab:", vocab)

Vocab: ['🐈', '🐯', '🐱', '🐩', '🐱', '🐶', '🐶', '🐈', '🐶', '🐩', '🐈', '🐶', '🐶', '🐈', '🐈', '🐯', '🐶', '🐈', '🐈', '🐈', '🐈', '🐶', '🐶', '🐯', '🐈', '🐩', '🐱', '🐩', '🐶', '🐩', '🐶']


In [85]:
# Unique tokens
set(vocab)

{'🐈', '🐩', '🐯', '🐱', '🐶'}

In [86]:
# Number of unique tokens, aka cardinality
v = len(set(vocab))
print("Cardinality of vocab:", v)

Cardinality of vocab: 5


In [87]:
# A default dict of default dicts; inner default dict is probability
cond_prob = defaultdict(lambda: defaultdict(float))

for label in labels:
    
    label_words = []
    for doc in train:
         # For a given label, get a list of all the tokens for all the docs 
        if doc.label == label:
            label_words.extend(doc.words)

    for word in vocab:
        # Find conditional probability: word count / total count
        cond_prob[label][word] = label_words.count(word) / len(label_words) 

cond_prob

defaultdict(<function __main__.<lambda>()>,
            {'cat': defaultdict(float,
                         {'🐈': 0.5384615384615384,
                          '🐯': 0.15384615384615385,
                          '🐱': 0.15384615384615385,
                          '🐩': 0.07692307692307693,
                          '🐶': 0.07692307692307693}),
             'dog': defaultdict(float,
                         {'🐈': 0.16666666666666666,
                          '🐯': 0.05555555555555555,
                          '🐱': 0.05555555555555555,
                          '🐩': 0.2222222222222222,
                          '🐶': 0.5})})

In [88]:
# Test that each label is a probability mass function (pmf). A pmf sums to 1
from math import isclose

for label in labels:
    assert isclose(sum(cond_prob[label].values()), 1)

Given a new document without a label,  calculate the proportional probabilities for each class
-------

$$ P(c | X) = P(c) •  \prod_{i=1}^n P(x_i | c)$$

In [89]:
import operator
from functools import reduce

def product(iterable):
    return reduce(operator.mul, iterable, 1)

In [97]:
# test = LabeledTextData(id_num=90, label=None, words="🐱".split())
# test = LabeledTextData(id_num=91, label=None, words="🐶 🐶".split()) 
# test = LabeledTextData(id_num=92, label=None, words="🐶 🐱".split())
# test = LabeledTextData(id_num=93, label=None, words="🐈 🐈 🐶 🐶 🐩 🐱 🐱".split())
# test = LabeledTextData(id_num=94, label=None, words="🐬 ".split()) # Out of sample prediction

prob_predicted = defaultdict(float)
for label in labels:
    # For each label, calculate the conditional probability based on the prior and the tokens that appear
    prob_predicted[label] = doc_priors[label] * product(cond_prob[label][t] for t in test.words)
    
print(*dict(prob_predicted).items(), sep='\n')

('cat', 0.0)
('dog', 0.0)


Pick the winning class
----

In [91]:
from operator import itemgetter

In [92]:
# Naive
label, prob = max(prob_predicted.items(),
                  key=itemgetter(1))
print("The predicted class is:", label)

The predicted class is: cat


<br>
<br> 
<br>

----

In [93]:
# Handle ties and fall back to document priors if winning probability is zero
label, prob = max(prob_predicted.items(),
                  key=itemgetter(1))
if prob > 0:
    print("The predicted class is: ", end="")
    print(*(k for k, v in prob_predicted.items() if v == prob))
else:
    label, prob = max(doc_priors.items(),
                      key=itemgetter(1))
    print("The predicted class is:", label)

The predicted class is: cat


Ideas for Improvement
-----

1. Switch from human-readable data class to matrix storage
    - Simpler
    - Faster (vectorized operations and fewer passes through the data)
    

Summary
------

- Naive Bayes (NB) is a simple and powerful algorithm for text classification
- To apply NB, follow a step-by-step process to calculate each probability
- Python's Standard Library has functions to write elegant and performant code

Bonus Material
----

<center><img src="images/bayesian_evol.png" width="75%"/></center>

Further Study
------

- [Data Science Handbook on Naive Bayes](https://jakevdp.github.io/PythonDataScienceHandbook/05.05-naive-bayes.html)
- https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/
- http://www.statsoft.com/textbook/naive-bayes-classifier (sorry about the blue front)
- Read [Text classification and Naive Bayes](https://web.stanford.edu/class/cs124/lec/naivebayes.pdf)   
- Read [Naive Bayes blogpost](http://sebastianraschka.com/Articles/2014_naive_bayes_1.html)
- https://towardsdatascience.com/naive-bayes-classifier-81d512f50a7c


<center><img src="https://imgs.xkcd.com/comics/modified_bayes_theorem.png" width="75%"/></center>